# Setting up Model

In [11]:
class Intent:
    name = ""
    keywords = []
    sentence = []
    answer = None
    entity = None

class Answer:
    uri = ""
    answer_template = None
    answer_property = ""
    refers_to = None
    
    def __str__(self):
        return f"{self.uri}"

print("Clases cargadas")

Clases cargadas


# Loading data into clases

In [15]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDF, FOAF
from rdflib import Namespace, Literal
import csv



intents_name = []

sentences = []

with open("data/intents_raw.csv", "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f, delimiter=",")
    for row in csv_reader:
        # print(row)
        if row[0] not in intents_name:
            intents_name.append(row[0])  
            
        sentences.append([row[0],row[1]])
    f.close()

entities = []


with open("data/intents.csv", "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f, delimiter=",")
    for row in csv_reader:
        entities.append([row[0], row[1]])
    f.close()


labels = []

with open("data/intents_label.csv", "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f, delimiter=",")
    for row in csv_reader:
        # print(row)
        labels.append([row[0], row[1]])
    f.close()

answers = []
with open("data/intents_answer.csv", "r", encoding="utf-8") as f:
    csv_reader = csv.reader(f, delimiter=",")
    for row in csv_reader:
        # print(row)
        aux_answer =  Answer()
        aux_answer.uri = row[0]+"Answer"
        
        if len(row[1]) >0:
            aux_answer.answer_template = row[1]
        elif len(row[3]) >0:
            aux_answer.refers_to = row[3]
        
        aux_answer.answer_property = row[3]

        answers.append([row[0], aux_answer])
    f.close()


intents = []
   
for name in intents_name:
    aux_intent = Intent()
    aux_intent.name = name
    aux_sentences = []
    aux_keywords = []
    for sentence in sentences:
        if name == sentence[0]:
            aux_sentences.append(sentence[1])
    for label in labels:
         if name == label[0]:
            aux_keywords.append(label[1])
            
    for entity in entities:
        if name == entity[0]:
            aux_intent.entity = entity[1]
            
    for answer in answers:
         if name ==answer[0]:
            aux_intent.answer = answer[1]
    
    
    aux_intent.sentence = aux_sentences
    aux_intent.keywords = aux_keywords
    
    intents.append(aux_intent)


for intent in intents:
    print(intent.name)
    print(f"[{intent.answer}]")
    print(f"({intent.entity})")
    for sentence in intent.sentence:
        print(">>>>", sentence)
    for keyword in intent.keywords:
        print(">>>>+++", keyword)


ObtenerInformacion
[ObtenerInformacionAnswer]
(http://127.0.0.1/ockb/ontology/Course)
>>>> De que trata el mooc?
>>>> Quiero informacion del curso de emprendimiento
>>>> Muestrame un resumen del mooc?
>>>> Breve introducción al curso
>>>> que es emprendimiento
>>>> De que se trata el curso?
>>>> De qué va el curso?
>>>> Me ayudas con información acerca del curso?
>>>>+++ trata
>>>>+++ informacion
>>>>+++ información
>>>>+++ resumen
>>>>+++ introducción
>>>>+++ dar
ObtenerFechas
[ObtenerFechasAnswer]
(http://127.0.0.1/ockb/ontology/Course)
>>>> Cuáles son las fechas importantes del curso?
>>>> Fechas clave del curso
>>>> Que fechas debo tomar en cuenta
>>>> fechas de inicio y fin
>>>> Cuándo comienza el curso?
>>>> Fechas importantes del curso de inteligencia artificial
>>>> Cuáles son las fechas importantes del curso de emprendimiento
>>>>+++ importantes
>>>>+++ clave
>>>>+++ fecha
ObtenerFechasInicio
[ObtenerFechasInicioAnswer]
(http://127.0.0.1/ockb/ontology/Course)
>>>> Cuándo inici

In [17]:

RESOURCE_BASE_URI = "http://127.0.0.1/ockb/resources/"
ockb = Namespace("http://127.0.0.1/ockb/ontology/")
g = Graph()


for intent in intents:
    intent_uri = URIRef(RESOURCE_BASE_URI+intent.name)
    print(intent_uri)
    g.add((intent_uri, RDF.type, ockb.Intent))
    g.add((intent_uri, ockb.intentName,Literal(intent.name)))
    
    answer_uri = URIRef(RESOURCE_BASE_URI+intent.answer.uri)
    print(f"[{answer_uri}]")
    g.add((intent_uri, ockb.hasAnswer,answer_uri))
    g.add((answer_uri, RDF.type,ockb.Answer))
    g.add((answer_uri, ockb.answerTemplate,Literal(intent.answer.answer_template)))
    g.add((answer_uri, ockb.answerProperty,URIRef(intent.answer.answer_property)))
    if intent.answer.refers_to is not None:
        g.add((answer_uri, ockb.refersTo,URIRef(intent.answer.refers_to)))
    
    if intent.entity is not None:
        g.add((intent_uri, ockb.requiresEntity,URIRef(intent.entity)))
    
    for sentence in intent.sentence:
        print(">>>>", sentence)
        g.add((intent_uri, ockb.trainingSentence,Literal(sentence)))
    for keyword in intent.keywords:
        print(">>>>+++", keyword)
        g.add((intent_uri, ockb.intentLabel,Literal(keyword)))
    
g.serialize(destination='intents.rdf')

# TODO add data types



# from rdflib import Literal,XSD
# Literal("01", datatype=XSD.int)

print("se ha finalizado")
    

http://127.0.0.1/ockb/resources/ObtenerInformacion
[http://127.0.0.1/ockb/resources/ObtenerInformacionAnswer]
>>>> De que trata el mooc?
>>>> Quiero informacion del curso de emprendimiento
>>>> Muestrame un resumen del mooc?
>>>> Breve introducción al curso
>>>> que es emprendimiento
>>>> De que se trata el curso?
>>>> De qué va el curso?
>>>> Me ayudas con información acerca del curso?
>>>>+++ trata
>>>>+++ informacion
>>>>+++ información
>>>>+++ resumen
>>>>+++ introducción
>>>>+++ dar
http://127.0.0.1/ockb/resources/ObtenerFechas
[http://127.0.0.1/ockb/resources/ObtenerFechasAnswer]
>>>> Cuáles son las fechas importantes del curso?
>>>> Fechas clave del curso
>>>> Que fechas debo tomar en cuenta
>>>> fechas de inicio y fin
>>>> Cuándo comienza el curso?
>>>> Fechas importantes del curso de inteligencia artificial
>>>> Cuáles son las fechas importantes del curso de emprendimiento
>>>>+++ importantes
>>>>+++ clave
>>>>+++ fecha
http://127.0.0.1/ockb/resources/ObtenerFechasInicio
[http